In [2]:
import pandas as pd 
import datetime as dt 
import numpy as np 
from sqlalchemy import create_engine
from credentials import password

### Read in Weather Data

In [3]:
csv = './csv/WeatherEvents_Jan2016-Dec2020.csv'

In [4]:
data = pd.read_csv(csv)
data.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


### Transform

In [5]:
data['StartTime(UTC)'] = pd.to_datetime(data['StartTime(UTC)'])
data['EndTime(UTC)'] = pd.to_datetime(data['EndTime(UTC)'])

In [6]:
data.dtypes

EventId                   object
Type                      object
Severity                  object
StartTime(UTC)    datetime64[ns]
EndTime(UTC)      datetime64[ns]
TimeZone                  object
AirportCode               object
LocationLat              float64
LocationLng              float64
City                      object
County                    object
State                     object
ZipCode                  float64
dtype: object

In [7]:
data['StartDate'] = pd.to_datetime(data['StartTime(UTC)']).dt.date
data['StartTime'] = pd.to_datetime(data['StartTime(UTC)']).dt.time
data['EndDate'] = pd.to_datetime(data['EndTime(UTC)']).dt.date
data['EndTime']=  pd.to_datetime(data['EndTime(UTC)']).dt.time

In [8]:
data.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,StartDate,StartTime,EndDate,EndTime
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-06,23:14:00,2016-01-07,00:34:00
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-07,04:14:00,2016-01-07,04:54:00
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-07,05:54:00,2016-01-07,15:34:00
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-08,05:34:00,2016-01-08,05:54:00
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-08,13:54:00,2016-01-08,15:54:00


In [25]:
weather = data[['EventId', 'Type', 'Severity', 'ZipCode', 'State', 'StartDate']].copy(deep = False)
weather.rename(columns={'EventID':'event_id', 'Type':'type', 'Severity':'severity', 'ZipCode':'zipcode', 'State':'us_state', 'StartDate':'startdate'})
weather.reset_index()
weather.head()

,EventId,Type,Severity,ZipCode,State,StartDate
0,W-1,Snow,Light,81149.0,CO,2016-01-06
1,W-2,Snow,Light,81149.0,CO,2016-01-07
2,W-3,Snow,Light,81149.0,CO,2016-01-07
3,W-4,Snow,Light,81149.0,CO,2016-01-08
4,W-5,Snow,Light,81149.0,CO,2016-01-08


In [26]:
weather.to_csv('./output/weather_data.csv')

### Read in Finance CSV

In [36]:
csv2 = './csv/delta.csv'

In [37]:
delta_stock = pd.read_csv(csv2)
delta_stock.rename(columns={'Date':'date', 'Open':'open_price', 'High':'high', 'Low':'low', 'Close':'close_price', 'Adj Close':'adj_c', 'Volume':'vol'}, inplace = True)
delta_stock.head()

,date,open_price,high,low,close_price,adj_c,vol
0,2016-01-06,47.790001,49.299999,47.630001,48.930000,44.468582,10375900
1,2016-01-07,48.389999,48.680000,46.740002,47.000000,42.714558,11301100
2,2016-01-08,47.480000,48.950001,46.490002,46.610001,42.360115,10850300
3,2016-01-11,46.990002,47.290001,45.570000,46.740002,42.478264,12096400
4,2016-01-12,47.070000,47.480000,45.820000,46.959999,42.678200,12163400


In [38]:
delta_stock.to_csv('./output/delta_stock.csv')

### Connect to Database 

In [31]:
connection_string = f"postgres:{password}@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{connection_string}')

In [32]:
engine.table_names()

['weather', 'delta']

### Upload Data to Database

In [40]:
weather.to_sql(name='weather', con=engine, if_exists='replace', index=False)

In [39]:
delta_stock.to_sql(name='delta', con=engine, if_exists='append', index=False)